In [1]:
import numpy as np
import pandas as pd
from siuba import *
import altair as alt
import altair_saver
from plotnine import *
from shared_utils import geography_utils
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [2]:
#import _clean_data

In [70]:
import _dla_utils

In [3]:
alt.themes.register("calitp_theme", styleguide.calitp_theme)
# enable
alt.themes.enable("calitp_theme")

ThemeRegistry.enable('calitp_theme')

In [4]:
%%html
<style>
@import url('https://fonts.googleapis.com/css?family=Lato');
</style>


In [5]:
#df = _clean_data.make_clean_data()
df= pd.read_parquet("dla_df.parquet")

In [6]:

#df['year'] = df['prepared_date'].dt.year.

* if i change the year to get rid of the ".0" using the code above, my functions do not recognize it and spit out an error. 
* if i do not change it, then the charts produce an error cannot read the data-type

In [7]:
df.sample(1)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,ftip_no,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,projectID,prepared_y,primary_agency_name
8048,Obligated,STPL,5146(012),Coalinga,2015-10-28,2015-10-28,2015-11-02,2015-11-02,2015-11-08,-29125.72,...,FRE130067,Forest St Btwn. 3rd And 5th Streets,Road Reconstruction (tc),4,NaT,NaT,CFCG,5146,2015.0,Coalinga


## Function to Count by Year

In [10]:
#from Tiffany's branch DLA functions
def count_all_years(df, groupedby=["dist"]):
    count_years = geography_utils.aggregate_by_geography(
        df, 
        group_cols = ["prepared_y", "dist"],
        sum_cols = ["total_requested", "ac_requested", "fed_requested"],
        mean_cols = ["total_requested", "ac_requested", "fed_requested"],
        nunique_cols = ["primary_agency_name", "prefix", "project_no", "project_location", "type_of_work"]
    ).sort_values(["prepared_y", "dist"], ascending=[False, True])
    
    count_years= count_years.rename(columns={"ac_requested_x": "ac_requested_sum",
                                "fed_requested_x": "fed_requested_sum",
                                "total_requested_x": "total_requested_sum",
                                "ac_requested_y": "ac_requested_mean",
                                "fed_requested_y": "fed_requested_mean",
                                "total_requested_y": "total_requested_mean",
                                "status": "counts",
                                "prefix": "unique_prefix",
                                "primary_agency_name": "unique_primary_agency_name",
                                "project_location": "unique_project_location",
                                "project_no":"unique_project_no",
                                "type_of_work":"unique_type_of_work"})

    return count_years

In [11]:
df1 = count_all_years(df)

In [12]:
df1

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
120,2021.0,0,0.000000e+00,-1.140550e+03,-1.140550e+03,0.000000,-1140.550000,-1140.550000,1.0,1.0,1.0,1.0,1.0
106,2021.0,1,6.484611e+06,4.167653e+06,1.217239e+07,67548.033437,43413.056250,126795.768542,13.0,9.0,86.0,85.0,49.0
107,2021.0,2,-1.135074e+06,6.084834e+06,7.705425e+06,-15134.315867,81131.117467,102739.006133,15.0,13.0,69.0,69.0,55.0
108,2021.0,3,1.035237e+08,1.089277e+08,2.378141e+08,417434.091169,439224.708387,958927.647621,34.0,45.0,191.0,198.0,164.0
109,2021.0,4,2.533793e+06,1.061114e+08,1.602708e+08,7541.050387,315807.822321,476996.483929,37.0,91.0,272.0,277.0,253.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2010.0,8,0.000000e+00,3.400000e+05,3.850000e+05,0.000000,340000.000000,385000.000000,1.0,1.0,1.0,1.0,1.0
118,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df2 = count_all_years((df>>filter(_.dist==4)))

In [14]:
df2

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
10,2021.0,4,2533792.93,1.061114e+08,1.602708e+08,7541.050387,3.158078e+05,4.769965e+05,37.0,91.0,272.0,277.0,253.0
9,2020.0,4,-15204495.21,2.150768e+08,3.816023e+08,-52429.293828,7.416440e+05,1.315870e+06,34.0,89.0,237.0,246.0,218.0
8,2019.0,4,-70929402.87,3.148522e+08,3.196755e+08,-176881.303915,7.851677e+05,7.971958e+05,41.0,93.0,322.0,333.0,300.0
0,2018.0,4,-19612828.51,2.861540e+08,3.740390e+08,-43200.062797,6.302951e+05,8.238745e+05,51.0,97.0,344.0,369.0,315.0
1,2017.0,4,-12268568.69,2.800907e+08,3.331716e+08,-33338.501875,7.611161e+05,9.053577e+05,43.0,98.0,293.0,320.0,270.0
2,2016.0,4,59687248.94,3.047821e+08,5.023206e+08,144172.098889,7.361886e+05,1.213335e+06,50.0,113.0,336.0,361.0,300.0
3,2015.0,4,68364142.00,2.399520e+08,4.486554e+08,152258.668151,5.344144e+05,9.992325e+05,48.0,109.0,374.0,394.0,300.0
4,2014.0,4,-21667366.00,1.783326e+08,2.292784e+08,-44491.511294,3.661861e+05,4.707975e+05,52.0,108.0,376.0,400.0,295.0
5,2013.0,4,928825.00,4.860823e+07,5.796681e+07,24442.763158,1.279164e+06,1.525442e+06,15.0,28.0,35.0,38.0,38.0
6,2012.0,4,0.00,3.652067e+06,1.037658e+07,0.000000,9.130167e+05,2.594145e+06,2.0,3.0,4.0,4.0,4.0


In [15]:
df3= count_all_years((df>>filter(_.prefix=="CML")))

In [16]:
df3

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
81,2021.0,3,-1551650.2,12795817.91,14581991.84,-43101.394444,3.554394e+05,4.050553e+05,1,19,27,27,27
82,2021.0,4,-2618000.0,12212570.69,11861596.27,-62333.333333,2.907755e+05,2.824190e+05,1,22,35,34,34
83,2021.0,6,126000.0,20537979.90,25601582.38,1000.000000,1.629998e+05,2.031872e+05,1,27,100,99,92
86,2021.0,7,0.0,1051428.42,4420340.15,0.000000,3.625615e+04,1.524255e+05,1,16,26,26,26
87,2021.0,8,0.0,2571684.65,3306330.34,0.000000,3.214606e+05,4.132913e+05,1,7,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2013.0,10,0.0,-19261.00,-191630.00,0.000000,-9.630500e+03,-9.581500e+04,1,2,2,2,2
55,2013.0,11,0.0,821000.00,927648.00,0.000000,2.736667e+05,3.092160e+05,1,3,3,3,3
58,2013.0,12,0.0,546814.00,620993.00,0.000000,1.822713e+05,2.069977e+05,1,1,3,3,1
49,2012.0,4,0.0,3842690.66,10576983.00,0.000000,1.280897e+06,3.525661e+06,1,2,3,3,3


In [17]:
count_all_years(df>>filter(_.primary_agency_name=="Humboldt County"))

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
8,2021.0,1,6553498.22,3433785.31,11132156.04,109224.970333,57229.755167,185535.934000,9,1,54,53,28
7,2020.0,1,-3136109.97,12204768.40,10262741.19,-37334.642500,145294.861905,122175.490357,9,1,59,60,32
6,2019.0,1,4946140.05,14222150.75,21295247.46,33647.211224,96749.324830,144865.628980,9,1,77,78,43
0,2018.0,1,177241.53,3015726.53,3600444.63,1036.500175,17635.827661,21055.231754,8,1,110,111,37
1,2017.0,1,0.00,5158080.59,5531542.79,0.000000,184217.163929,197555.099643,8,1,24,24,16
3,2016.0,1,0.00,4649799.00,5028074.27,0.000000,140903.000000,152365.886970,10,1,29,30,21
2,2015.0,1,0.00,4659366.97,5363881.78,0.000000,89603.210962,103151.572692,8,1,35,35,22
4,2014.0,1,0.00,4554254.42,4981763.78,0.000000,89299.106275,97681.642745,9,1,45,45,23
5,2013.0,1,0.00,100863.00,113931.00,0.000000,100863.000000,113931.000000,1,1,1,1,1


## Chart Part 1: Aggregate the Data

In [18]:
def part1_chart(df, col, aggregate_by=["dist"]):
    aggfunc = (
        geography_utils.aggregate_by_geography(
            df[df[aggregate_by].notna()], 
            group_cols = [aggregate_by],
            nunique_cols = [col]))       
    return aggfunc

In [19]:
part1_chart(df, "type_of_work", aggregate_by="primary_agency_name")

,primary_agency_name,type_of_work
0,Humboldt County,89
1,Mendocino County,54
2,Sacramento County,82
3,Sutter County,11
4,Contra Costa County,59
...,...,...
614,Palos Verdes Est,1
615,San Diego Metropolitan Transit System,1
616,Corte Madera,1
617,Hillsborough,1


## Find Top Nuniques

In [20]:
# #gives error message
# def find_top_test(df):   
#     cols= [{"prefix",
#            "agency",
#            "prepared_y",
#            "status_comment",
#            "project_location",
#            "type_of_work",
#            "seq",
#            "mpo",
#            "primary_agency_name"}]
#     for col in cols:
#         agg = (df >> count(_[col]) >> arrange(-_.n))
#         agg = (agg.rename(columns = {"n": f"test_{col}"}))
#         display(agg)
        
#     return agg

In [21]:
# find_top_test(df)

In [22]:
## want to create a function that gets a new dataframe that will hold all the counts and top twenty 
def find_top(df):
    
    cols= [["prefix",
           "prepared_y",
           "status_comment",
           "project_location",
           "type_of_work",
           "seq",
           "mpo",
           "primary_agency_name"]]
    
    tops_1 = (df
          >>count(_.prefix)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_prefix"})
    tops_2 = (df
          >>count(_.prepared_y)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_prepared_y"})
    tops_3 = (df
          >>count(_.status_comment)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_status_comment"})
    tops_4 = (df
          >>count(_.project_location)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_project_location"})
    tops_5 = (df
          >>count(_.type_of_work)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_type_of_work"})
    tops_6 = (df
          >>count(_.mpo)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_mpo"})
    tops_7 = (df
          >>count(_.primary_agency_name)
          >>arrange(-_.n)).head(20
                               ).reset_index(drop=True
                                            ).rename(columns={"n": "unique_primary_agency_name"})
    
    
    test = tops_1.join(tops_2, how='outer')
    test2 = test.join(tops_3, how='outer')
    test3 = test2.join(tops_4, how='outer')
    test4 = test3.join(tops_5, how='outer')
    test5 = test4.join(tops_6, how='outer')
    tops = test5.join(tops_7, how='outer')
                       
    return tops

In [23]:
tops = find_top(df)
tops

,prefix,unique_prefix,prepared_y,unique_prepared_y,status_comment,unique_status_comment,project_location,unique_project_location,type_of_work,unique_type_of_work,mpo,unique_mpo,primary_agency_name,unique_primary_agency_name
0,CML,3293,2018.0,2827.0,Authorized,19399,San Francisco Bay Area,65,Bridge Replacement (tc),679,SCAG,4988,Humboldt County,627
1,HSIPL,3282,2014.0,2724.0,..,194,Various Locations,31,Bridge Replacement,519,MTC,3236,Los Angeles,490
2,STPL,2779,2019.0,2665.0,Prog Code Z400,108,Sacog Region,29,FTA Transfer,462,NON-MPO,2592,Fresno,459
3,ER,1590,2015.0,2575.0,Prog Code M400,73,Golden Gate Bridge,27,Permanent Restoration,282,SACOG,1865,Los Angeles County,456
4,BRLO,1402,2016.0,2505.0,Prog Code Z230,47,Throughout Kern County,22,Road Rehabilitation,269,CFCG,1404,Santa Cruz County,446
5,BRLS,904,2020.0,2350.0,Prog Code Z003,30,Citywide,18,Bridge Preventive Maintenance,175,KCOG,998,El Dorado County,297
6,ATPL,587,2017.0,2333.0,Prog Code M40E,26,Various Locations (see Comments),14,Emergency Opening,153,STANCOG,759,Kern County,283
7,RPSTPL,576,2021.0,1917.0,Prog Code M0E3,22,Various Locations In San Diego County,14,Pavement Rehabilitation,120,AMBAG,745,Fresno County,271
8,HPLUL,332,2013.0,193.0,Prog Code M23E,17,County Of Madera,13,Seismic Retrofit,119,SJCG,515,Bakersfield,270
9,SRTSL,331,2012.0,7.0,Prog Code Z301,14,FTA transfer,13,Bridge Rehabilitation,110,TCAG,489,San Joaquin County,266


In [30]:
def basic_bar_chart(df, col, aggregate_by):
    
    chart = (alt.Chart(df)
             .mark_bar()
             .encode(
                 x=alt.X(aggregate_by, title=labeling(aggregate_by), sort=('-y')),
                 y=alt.Y(col, title=labeling(col)),
                 #column = "payment:N",
                 color = alt.Color(col,
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=(labeling(col)))
                                  ))
             .properties( 
                          title=f"Highest {labeling(col)} by {labeling(aggregate_by)}")
    )


    #chart.save(f"./chart_outputs/{col}_{aggfunc}_by_{aggregate_by}.png")
    
    return chart



In [31]:
basic_bar_chart(tops, "unique_prepared_y", "prepared_y")

alt.Chart(...)

In [32]:
df2.sample()

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
11,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
basic_bar_chart((df2),
            "unique_prefix",
            "prepared_y")

alt.Chart(...)

In [35]:
basic_bar_chart(tops, "unique_primary_agency_name", "primary_agency_name")

alt.Chart(...)

In [36]:
def basic_scatter_chart(df, col, aggregate_by, colorcol):
    
    chart = (alt.Chart(df)
             .mark_circle(size=60)
             .encode(
                 x=alt.X(aggregate_by, title=labeling(aggregate_by)),
                 y=alt.Y(col, title=labeling(col)),
                 #column = "payment:N",
                 color = alt.Color(colorcol,
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=(labeling(col)))
                                  ))
             .properties( 
                          title=f"Highest {labeling(col)} by {labeling(aggregate_by)}")
    )


    #chart.save(f"./chart_outputs/{col}_{aggfunc}_by_{aggregate_by}.png")
    
    return chart


In [37]:
basic_scatter_chart(tops, "unique_primary_agency_name", "primary_agency_name", "unique_primary_agency_name")

alt.Chart(...)

In [38]:
basic_scatter_chart(df1, "unique_primary_agency_name", "total_requested_sum", "dist").interactive()

alt.Chart(...)

In [49]:
def basic_line_chart(df, x_col, y_col):
    
    chart = (alt.Chart(df)
             .mark_line()
             .encode(
                 x=alt.X(x_col, title=labeling(x_col)),
                 y=alt.Y(y_col, title=labeling(y_col))
                 # ,
    #              color = alt.Color(y_col,
    #                               scale=alt.Scale(
    #                                   range=altair_utils.CALITP_SEQUENTIAL_COLORS),
    #                                   legend=alt.Legend(title=(labeling(y_col)))
                                   )
              )
    #          .properties( 
    #                       title=f"{labeling(y_col)} by {labeling(x_col)}")
    # )


    #chart.save(f"./chart_outputs/{col}_{aggfunc}_by_{aggregate_by}.png")
    
    return chart

In [1]:
basic_line_chart(df1, "prepared_y:O", "total_requested_sum")

NameError: name 'basic_line_chart' is not defined

In [51]:
basic_line_chart(df1, "prepared_y", "ac_requested_sum")

alt.Chart(...)

In [41]:
df1

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
120,2021.0,0,0.000000e+00,-1.140550e+03,-1.140550e+03,0.000000,-1140.550000,-1140.550000,1.0,1.0,1.0,1.0,1.0
106,2021.0,1,6.484611e+06,4.167653e+06,1.217239e+07,67548.033437,43413.056250,126795.768542,13.0,9.0,86.0,85.0,49.0
107,2021.0,2,-1.135074e+06,6.084834e+06,7.705425e+06,-15134.315867,81131.117467,102739.006133,15.0,13.0,69.0,69.0,55.0
108,2021.0,3,1.035237e+08,1.089277e+08,2.378141e+08,417434.091169,439224.708387,958927.647621,34.0,45.0,191.0,198.0,164.0
109,2021.0,4,2.533793e+06,1.061114e+08,1.602708e+08,7541.050387,315807.822321,476996.483929,37.0,91.0,272.0,277.0,253.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,2010.0,8,0.000000e+00,3.400000e+05,3.850000e+05,0.000000,340000.000000,385000.000000,1.0,1.0,1.0,1.0,1.0
118,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Issues/what to look for:

* get a df of information not grouped by year with 
    - total requested
    - ac requested 
    - fed requested
* fix name for charts of find_top 


## Labeling 

In [52]:
LABEL_DICT = {
    "prepared_y": "Year",
    "dist": "District",
    "total_requested": "Total Requested",
    "fed_requested":"Fed Requested",
    "ac_requested": "Advance Construction Requested",
    "nunique":"Number of Unique",
    "project_no": "Project Number"
}


In [53]:
# Tiffany's function for changing labels
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        word = word.replace('n_', 'Number of ').title()
        word = word.replace('unique_', "Number of Unique ").title()
        word = word.replace('_', ' ').title()
    
    return word

In [54]:
print(labeling("primary_agency_name"))

Primary Agency Name


## Chart Function

### Old Function with filtering in the function

In [55]:

def basic_agg(df, col, aggregate_by=["dist"]):
    df1 = ((df >> group_by(_[aggregate_by]) >> summarize(n=_[col].nunique()) >> arrange(-_.n)).head(20))
    chart = (alt.Chart(df1)
             .mark_bar()
             .encode(
                 x=alt.X(aggregate_by, title=f"{labeling(aggregate_by)}"),
                 y=alt.Y("n", title=f"Number of Unique {labeling(col)}"),
                 #column = "payment:N",
                 color = alt.Color("n",
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=f"{(labeling(col))}")
                                  ))
             .properties( 
                          title=f"Number of Unique {labeling(col)} by {labeling(aggregate_by)}")
    )


    #chart.save(f"./chart_outputs/{col}_by_{aggregate_by}.png")

    return chart

In [56]:
basic_agg(df, "prefix", aggregate_by="primary_agency_name")

alt.Chart(...)

## Function for Selecting Calculating Method

In [57]:
def calculate_data(df, col, aggregate_by=["dist"], aggfunc="sum"):
    df1 = (df.groupby(aggregate_by)
          .agg({col: aggfunc})
          .reset_index()
          .sort_values(col, ascending=False)
         )
    #df = df[df[col] > 0]
    df1 = (df1>>arrange(-_[col])).head(20)
    return df1

In [58]:
calculate_data(df, "total_requested", aggregate_by="dist", aggfunc="sum")

,dist,total_requested
7,7,4.264832e+09
12,12,2.858146e+09
4,4,2.820382e+09
8,8,2.589842e+09
6,6,1.597301e+09
3,3,1.561961e+09
11,11,1.159055e+09
10,10,1.069134e+09
5,5,4.926226e+08
2,2,2.288827e+08


In [59]:
# #calcualte_sum2 might be more versatile
# def calculate_sum(df, col, agg_col):
#     dftest = geography_utils.aggregate_by_geography(
#         df, 
#         group_cols = [agg_col],
#         sum_cols = [col])
#     dftest = (dftest>>arrange(-_[col])).head(20)
#     #dftest = (dftest>>filter(_[col] >0))
#     return dftest


In [60]:
# calculate_sum(df, "total_requested", agg_col="dist")

## Adding to Charting Function

In [61]:
def test_chartfunc_sum(df, col, aggregate_by, aggfunc):
    
    df1 = (calculate_data(df, col, aggregate_by, aggfunc))
    
    chart = (alt.Chart(df1)
             .mark_bar()
             .encode(
                 x=alt.X(aggregate_by, title=labeling(aggregate_by), sort=('-y')),
                 y=alt.Y(col, title=labeling(col)),
                 #column = "payment:N",
                 color = alt.Color(col,
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),
                                      legend=alt.Legend(title=(labeling(col)))
                                  ))
             .properties( 
                          title=f"Highest {labeling(col)} {labeling(aggfunc)}s by {labeling(aggregate_by)}")
    )


    #chart.save(f"./chart_outputs/{col}_{aggfunc}_by_{aggregate_by}.png")
    
    return chart



In [62]:
test_chartfunc_sum(df, "total_requested", aggregate_by="type_of_work", aggfunc="mean")

alt.Chart(...)

In [63]:
test_chartfunc_sum(df, "prefix", aggregate_by="primary_agency_name", aggfunc="nunique")

alt.Chart(...)

In [64]:
#making sure it is right
df>>group_by(_.primary_agency_name)>> summarize(n=_.prefix.nunique()) >> arrange(-_.n)

,primary_agency_name,n
276,Los Angeles,34
277,Los Angeles County,31
434,San Bernardino Associated Governments,29
71,Caltrans,27
534,Stockton,23
...,...,...
591,Weed,1
595,Western Shasta Resource Conservation District,1
602,Willits,1
603,Willows,1


## Grouping Charts

In [65]:
#groupby_col_x axis = prepared_y
#agg_by_col = dist or mpo 
#sum_col= primary_agency_name or prefix
 
def group_chart_nunique(df, groupby_col_x, agg_by_col, sum_col):
    dist_years1 = (df >> group_by(_[groupby_col_x], _[agg_by_col]) 
                   >> summarize(n=_[sum_col].nunique()) 
                   >> arrange(-_[groupby_col_x]))

    chart = alt.Chart(dist_years1).mark_bar().encode(
            column=(f"{agg_by_col}:N"),
            x=alt.X((f"{groupby_col_x}:O"), title=labeling(groupby_col_x)),
            y=alt.Y('n:Q', title=(f"Number of Unique {labeling(sum_col)}")),
            color = alt.Color((f"{agg_by_col}:N"), 
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),  
                                   legend=alt.Legend(title=(labeling(sum_col)))
                                  )
                                  )
    #chart.save(f"./chart_outputs/grouped_{agg_by_col}_by_{groupby_col_x}.png")
    
    return chart


In [66]:
def group_chart_nunique_2(df, groupby_col_x, agg_by_col, sum_col):
    #without function
    # dist_years1 = (df >> group_by(_[groupby_col_x], _[agg_by_col]) 
    #                >> summarize(n=_[sum_col].nunique()) 
    #                >> arrange(-_[groupby_col_x]))

    chart = alt.Chart(df).mark_bar().encode(
            column=(f"{agg_by_col}:N"),
            x=alt.X((f"{groupby_col_x}:O"), title=labeling(groupby_col_x)),
            y=alt.Y('n:Q', title=(f"Number of Unique {labeling(sum_col)}")),
            color = alt.Color((f"{agg_by_col}:N"), 
                                  scale=alt.Scale(
                                      range=altair_utils.CALITP_SEQUENTIAL_COLORS),  
                                   legend=alt.Legend(title=(labeling(sum_col)))
                                  )
                                  )
   # chart.save(f"./chart_outputs/grouped_{agg_by_col}_by_{groupby_col_x}.png")
    
    return chart

In [67]:
dist_years1 = (df >> group_by(_.prepared_y, _.mpo)
               >> summarize(n=_.prefix.nunique()) 
               >> arrange(-_.prepared_y))

In [68]:
dist_years1

,prepared_y,mpo,n
173,2021.0,AMBAG,11
174,2021.0,BCAG,9
175,2021.0,CFCG,10
176,2021.0,FCOG,1
177,2021.0,KCAG,6
...,...,...,...
1,2011.0,MTC,1
2,2011.0,NON-MPO,1
3,2011.0,SCAG,2
4,2011.0,STANCOG,1


In [69]:
df2

,prepared_y,dist,ac_requested_sum,fed_requested_sum,total_requested_sum,ac_requested_mean,fed_requested_mean,total_requested_mean,unique_prefix,unique_primary_agency_name,unique_project_location,unique_project_no,unique_type_of_work
10,2021.0,4,2533792.93,1.061114e+08,1.602708e+08,7541.050387,3.158078e+05,4.769965e+05,37.0,91.0,272.0,277.0,253.0
9,2020.0,4,-15204495.21,2.150768e+08,3.816023e+08,-52429.293828,7.416440e+05,1.315870e+06,34.0,89.0,237.0,246.0,218.0
8,2019.0,4,-70929402.87,3.148522e+08,3.196755e+08,-176881.303915,7.851677e+05,7.971958e+05,41.0,93.0,322.0,333.0,300.0
0,2018.0,4,-19612828.51,2.861540e+08,3.740390e+08,-43200.062797,6.302951e+05,8.238745e+05,51.0,97.0,344.0,369.0,315.0
1,2017.0,4,-12268568.69,2.800907e+08,3.331716e+08,-33338.501875,7.611161e+05,9.053577e+05,43.0,98.0,293.0,320.0,270.0
2,2016.0,4,59687248.94,3.047821e+08,5.023206e+08,144172.098889,7.361886e+05,1.213335e+06,50.0,113.0,336.0,361.0,300.0
3,2015.0,4,68364142.00,2.399520e+08,4.486554e+08,152258.668151,5.344144e+05,9.992325e+05,48.0,109.0,374.0,394.0,300.0
4,2014.0,4,-21667366.00,1.783326e+08,2.292784e+08,-44491.511294,3.661861e+05,4.707975e+05,52.0,108.0,376.0,400.0,295.0
5,2013.0,4,928825.00,4.860823e+07,5.796681e+07,24442.763158,1.279164e+06,1.525442e+06,15.0,28.0,35.0,38.0,38.0
6,2012.0,4,0.00,3.652067e+06,1.037658e+07,0.000000,9.130167e+05,2.594145e+06,2.0,3.0,4.0,4.0,4.0


## Plot Nine

In [ ]:
def prefix_all_agencies(df, prefix_unique):
    
    # graphs 
    prefixes = df[df.prefix== prefix_unique]
    
    prefix_count_num = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(50)
    
    prefix_count = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(20)
    
    display(Markdown(f"**The number of agencies using {prefix_unique} is {len(prefix_count_num)}**"))
    
    # for the table- using one as some agencies only have one entry
    display(df[(df.prefix == prefix_unique)].sample(1))
    
    
    ax1 = (prefix_count
            >> ggplot(aes("primary_agency_name", "n", fill="primary_agency_name")) 
                + geom_col() 
                + theme(axis_text_x = element_text(angle = 45 , hjust=1))
                + labs(title='Top Agencies using Prefix', x='Agency', y='Number of Obligations', fill="Agency")
                + theme_538()
                + theme(plot_background=element_rect(fill=backgroundColor, color=backgroundColor),
                     panel_background=element_rect(fill=backgroundColor, color=backgroundColor),
                     panel_grid_major_y=element_line(
                        color=axisColor, linetype='solid', size=1),
                     panel_grid_major_x=element_blank(),
                     figure_size=(7.0, 4.4),
                     title=element_text(weight="bold", size=font_size, 
                                        family=font, color=blackTitle),
                     axis_title=element_text(family=labelFont, size=12, color=guideTitleColor),
                     axis_text=element_text(family=labelFont, size=10, color=guideLabelColor, 
                                            margin={'r': 4}
                                           ),
                     axis_title_x=element_text(margin={'t': 10}),
                     axis_title_y=element_text(margin={'r': 10}),
                     legend_title=element_text(font=labelFont, size=14, color=blackTitle, 
                                               margin={'b': 10}),
                     legend_text=element_text(font=labelFont, size=11, color=blackTitle, 
                                              margin={'t': 5, 'b': 5, 'r': 5, 'l': 5}),
                    )
            )    
    #ax1 = preset_plotnine_config(ax1)
    return ax1
  

In [ ]:
prefixes = df[df.prefix== "CML"]
    
prefix_count_num = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(50)
    
prefix_count = (prefixes >> count(_.primary_agency_name) >> arrange(-_.n)).head(20)

In [ ]:
from shared_utils import calitp_color_palette as cp

In [ ]:
# ax1 = (prefix_count
#             >> ggplot(aes("primary_agency_name", "n", fill="primary_agency_name")) 
#                 + geom_col() 
#                 + theme(axis_text_x = element_text(angle = 45 , hjust=1))
#                 + labs(title='Top Agencies using Prefix', x='Agency', y='Number of Obligations', fill="Agency")
#             )

# chart = (styleguide.preset_plotnine_config(ax1)
#          + scale_fill_manual(values=cp.CALITP_SEQUENTIAL_COLORS)
#         )

# chart

# Misc.

In [ ]:
# def totalfunds_chart_(df, aggregate_by=[], col):
#      aggfunc = (
#         geography_utils.aggregate_by_geography(
#             df[df[aggregate_by].notna()], 
#             group_cols = [aggregate_by],
#             sum_cols = [col],
#             nunique_cols = [col]

            
    
#     test= (df
#            >>group_by(_.primary_agency_name)
#            >>summarize(Total_Funds = _[col].sum())
#            >>arrange(-_.Total_Funds))
#     #test.rename(columns={'primary_agency_name': 'Primary Agency Name', 'col': 'Sum Funds'}, inplace=True)
#     test.style.format(precision=2, na_rep='MISSING', thousands=",")
#     chart = (alt.Chart(test.head(20))
#              .mark_bar()
#              .encode(
#                  x=alt.X("primary_agency_name", title="col"),
#                  y=alt.Y("Total Funds", title="Total Funds Obligated"),
#                  #column = "payment:N",
#                  color = alt.Color("primary_agency_name", 
#                                   scale=alt.Scale(
#                                       range=altair_utils.FIVETHIRTYEIGHT_DIVERGING_COLORS
#                                   )
#                                   )
#              )
#             .properties( 
#                            title="District 7 Agencies with the Most Total Funds Obligated")

#     )
#     return chart